## Machine Learning Tutorials with Watson Machine Learning
### Part 1 - Linear Regression pt 1

In [1]:
import random
import numpy as np
import pylab as pl

from pyspark.sql.types import *
from pyspark.ml.linalg import SparseVector, VectorUDT
from pyspark.ml import Pipeline, Model
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

import urllib3, requests, json

In [20]:
x = [num / 1.0 for num in range(100)]
y = range(100)

for i in range(0,100):
    y[i] = x[i] + random.random() * random.randint(-5,5)

xytuple = zip(y,x)


schema = StructType([
    StructField("label", DoubleType(), True),
    StructField("x", DoubleType(), True),
])

df = sqlContext.createDataFrame(xytuple, schema)
trainingdf, test1df, test2df = df.randomSplit([0.8, 0.1, 0.1], 24)

In [21]:
featuresvector = VectorAssembler(inputCols=["x"], outputCol="features")
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

pipeline = Pipeline(stages =[featuresvector, lr])
pipelinemodel = pipeline.fit(trainingdf)

In [22]:
predictions = pipelinemodel.transform(test1df)
predictions.select('x','label','prediction').show()

In [ ]:
service_path = 'https://ibm-watson-ml.mybluemix.net'
instance_id = '437c58bc-4c25-4002-98de-e822fa4ec797'
username = 'cfd3c4a9-159e-476d-b097-1cd133d884d7'
password = 'ca465385-c8e6-4d35-8e82-0e47e6d399bf'

ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

model_artifact = MLRepositoryArtifact(pipelinemodel, name='xy linear regression', training_data=trainingdf)
saved_model = ml_repository_client.models.save(model_artifact)

print saved_model

In [ ]:
service_path = 'https://ibm-watson-ml.mybluemix.net'
partner_instance_id = '***'
partner_username = '***'
partner_password = '***'
partner_saved_model_uid = '***'

ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(partner_username, partner_password)

In [ ]:
loadedModel = ml_repository_client.models.get(partner_saved_model_uid)
print str(loadedModel.name)

In [ ]:
loadedModel.model_instance().transform(test2df).select('x','label','prediction').show()

In [ ]:
plot 

